<img style="float:left" width="70%" src="../../documentacion/imagenes/escudo_COLOR_1L_DCHA.png">
<img style="float:right" width="10%" src="../../documentacion/imagenes/PythonLogo.svg">
<br style="clear:both;">


<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px;">7. Machine Learning</h2>


 
## Estudiante:

- Jose Luis Perez Gomez

In [7]:
%run Importaciones.ipynb
%run Variables_Clases.ipynb

#

In [13]:
experimento_1_nor = pd.read_csv('../datos/experimento_1.csv', sep=',')
experimento_2_nor = pd.read_csv('../datos/experimento_2.csv', sep=',')
experimento_3_nor = pd.read_csv('../datos/experimento_3.csv', sep=',')
experimento_4_nor = pd.read_csv('../datos/experimento_4.csv', sep=',')
experimentos_all = pd.read_csv('../datos/experimento_all.csv', sep=',')

experimentos_nor = [experimento_1_nor, experimento_2_nor, experimento_3_nor,experimento_4_nor]



In [11]:
experimento_1_nor.head()

,Timestamp,Attention,Meditation,Delta,Theta,LowAlpha,HighAlpha,LowBeta,HighBeta,LowGamma,HighGamma,Key
0,54127.0,0.838384,0.474747,0.009006,0.011185,0.006130,0.006843,0.001222,0.030424,0.039161,0.036669,4.0
1,55056.0,0.777778,0.595960,0.001631,0.003823,0.016003,0.021995,0.000523,0.019995,0.024421,0.019984,0.0
2,56038.0,0.656566,0.747475,0.037205,0.011676,0.023153,0.014381,0.001926,0.046070,0.052352,0.148078,0.0
3,57043.0,0.606061,0.868687,0.020813,0.009838,0.011090,0.008470,0.002168,0.032221,0.051796,0.042572,0.0
4,58203.0,0.363636,0.838384,0.007665,0.059127,0.020394,0.015500,0.003289,0.005862,0.014763,0.010519,0.0


# holdout

Conjunto de entrenamiento: 81% del total
Conjunto de validación: 9% del total
Conjunto de prueba: 10% del total

In [54]:
tests_holdout= []
matriz_confusion =0


def evaluar_holdout(dataframe):
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
    
    # Se dividen los datos. X_train_val sera el 90% de los datos y X_test el 10%
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=SEED, train_size=TRAIN)

    # Se dividen los datos. X_train sera el 90% de los datos de X_train_val y X_val el restante 10%
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=SEED, train_size=TRAIN)
    
    tests_holdout.append(X_test)
    tests_holdout.append(y_test)
    
    # Entreno el modelo
    model_KNN.fit(X_train, y_train)
    model_TREE.fit(X_train, y_train)
    model_RANDOM.fit(X_train, y_train)
    
    # Evaluo el modelo con X_val
    y_predict_knn = model_KNN.predict(X_val)
    y_predict_tree = model_TREE.predict(X_val)
    y_predict_random = model_RANDOM.predict(X_val)
    
    # Evaluo el rendimiento que ha tenido Validation 
    knn_ac = accuracy_score(y_val, y_predict_knn)
    tree_ac = accuracy_score(y_val, y_predict_tree)
    random_ac = accuracy_score(y_val, y_predict_random)
    
    matriz_confusion = confusion_matrix(y_val, y_predict_knn)
    
    
    return ((knn_ac), (tree_ac), (random_ac))
    
    

In [55]:
display(tests_holdout)

[]

In [56]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_holdout(exp))

datos_holdout = pd.DataFrame(modelos_all, columns=['Knn_holdout', 'Tree_holdout', 'Random_holdout'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_holdout = datos_holdout.rename(index=nuevos_indices)

In [57]:
len(tests_holdout)

8

In [58]:
display(datos_holdout)

,Knn_holdout,Tree_holdout,Random_holdout
Exp_1,0.916667,0.916667,0.666667
Exp_2,0.714286,0.857143,0.500000
Exp_3,0.944444,0.722222,0.555556
Exp_4,0.842105,0.736842,0.263158


 # k-fold cross-validation

In [23]:
#ACC_SCORER

def evaluar_cross_val(dataframe):
    
    knn_means = []
    tree_means = []
    random_means =[]
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
     
    # Calculo las  metricas       
    knn = cross_val_score(model_KNN, X, y, cv=CV, scoring=ACC_SCORER)      
    tree = cross_val_score(model_TREE, X, y, cv=CV, scoring=ACC_SCORER)  
    random = cross_val_score(model_RANDOM, X, y, cv=CV, scoring=ACC_SCORER)

    # Calculo la media de las metricas anteriores
    knn_mean = knn.mean()
    tree_mean = tree.mean()
    random_mean = random.mean()

    return ((knn_mean), (tree_mean), (random_mean))
  


In [24]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_cross_val(exp))

datos_cross_val = pd.DataFrame(modelos_all, columns=['Knn_cross_val', 'Tree_cross_val', 'Random_cross_val'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_cross_val = datos_cross_val.rename(index=nuevos_indices)

In [25]:
display(datos_cross_val)

,Knn_cross_val,Tree_cross_val,Random_cross_val
Exp_1,0.920238,0.836905,0.538095
Exp_2,0.866071,0.779464,0.500893
Exp_3,0.837222,0.761667,0.464444
Exp_4,0.758636,0.685000,0.363636


# leave-one-out

In [26]:
#ACC_SCORER

def evaluar_loo(dataframe):
    
    knn_means = []
    tree_means = []
    random_means =[]
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
     
    # Calculo las  metricas       
    knn = cross_val_score(model_KNN, X, y, cv=LOO, scoring=ACC_SCORER)      
    tree = cross_val_score(model_TREE, X, y, cv=LOO, scoring=ACC_SCORER)  
    random = cross_val_score(model_RANDOM, X, y, cv=LOO, scoring=ACC_SCORER)

    # Calculo la media de las metricas anteriores
    knn_mean = knn.mean()
    tree_mean = tree.mean()
    random_mean = random.mean()

    return ((knn_mean), (tree_mean), (random_mean))

In [27]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_loo(exp))

datos_loo = pd.DataFrame(modelos_all, columns=['Knn_loo', 'Tree_loo', 'Random_loo'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_loo = datos_loo.rename(index=nuevos_indices)

In [28]:
display(datos_loo)

,Knn_loo,Tree_loo,Random_loo
Exp_1,0.930769,0.838462,0.507692
Exp_2,0.927152,0.867550,0.529801
Exp_3,0.923077,0.764103,0.512821
Exp_4,0.865385,0.769231,0.365385


In [29]:


df_concatenado = pd.concat([datos_holdout, datos_cross_val, datos_loo], axis=1)


display(df_concatenado)


,Knn_holdout,Tree_holdout,Random_holdout,Knn_cross_val,Tree_cross_val,Random_cross_val,Knn_loo,Tree_loo,Random_loo
Exp_1,1.000000,0.923077,0.461538,0.920238,0.836905,0.538095,0.930769,0.838462,0.507692
Exp_2,0.937500,0.875000,0.625000,0.866071,0.779464,0.500893,0.927152,0.867550,0.529801
Exp_3,0.800000,0.650000,0.550000,0.837222,0.761667,0.464444,0.923077,0.764103,0.512821
Exp_4,0.809524,0.714286,0.476190,0.758636,0.685000,0.363636,0.865385,0.769231,0.365385


In [30]:

#primer mayor valor en verde
#segundo mayor valor en amarillo 


def colorear_mayores(fila):
    mayores_indices = fila.nlargest(2).index  # Obtener los índices de los tres valores más grandes en la fila
    
    primer_mayor = mayores_indices[0]
    segundo_mayor = mayores_indices[1]
    
    estilos = []
    for columna in fila.index:

        if columna == primer_mayor:
            estilo = 'background-color: green'
        elif columna == segundo_mayor:
            estilo = 'background-color: yellow'
        else:
            estilo = ''
        estilos.append(estilo)
    
    return estilos

comparativa_ml = df_concatenado.style.apply(colorear_mayores, axis=1)

display(comparativa_ml)



,Knn_holdout,Tree_holdout,Random_holdout,Knn_cross_val,Tree_cross_val,Random_cross_val,Knn_loo,Tree_loo,Random_loo
Exp_1,1.000000,0.923077,0.461538,0.920238,0.836905,0.538095,0.930769,0.838462,0.507692
Exp_2,0.937500,0.875000,0.625000,0.866071,0.779464,0.500893,0.927152,0.867550,0.529801
Exp_3,0.800000,0.650000,0.550000,0.837222,0.761667,0.464444,0.923077,0.764103,0.512821
Exp_4,0.809524,0.714286,0.476190,0.758636,0.685000,0.363636,0.865385,0.769231,0.365385


# cross validation

In [31]:
def evaluar_cross_validation(dataframe):
    
    aciertos = []
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
    
    
    X1, X2, y1, y2 = train_test_split(X, y, random_state=SEED, train_size=0.5)

    
    y2_model = model_KNN.fit(X1, y1).predict(X2)
    y1_model = model_KNN.fit(X2, y2).predict(X1)
    aciertos.append(accuracy_score(y1, y1_model))
    aciertos.append(accuracy_score(y2, y2_model))
    media = np.mean(aciertos)
    aciertos.append(media)
 
    return aciertos

In [32]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_cross_validation(exp))   

datos_cross_validation = pd.DataFrame(modelos_all, columns=['Knn_cross_validation_A','Knn_cross_validation_B','Knn_cross_validation_Media'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_cross_validation = datos_cross_validation.rename(index=nuevos_indices)


In [33]:
display(datos_cross_validation)

,Knn_cross_validation_A,Knn_cross_validation_B,Knn_cross_validation_Media
Exp_1,0.830769,0.861538,0.846154
Exp_2,0.720000,0.684211,0.702105
Exp_3,0.835052,0.857143,0.846097
Exp_4,0.740385,0.625000,0.682692


# cross_val_predict

In [34]:
def evaluar_cross_val_predict(dataframe):
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
        
    predicciones = cross_val_predict(model_KNN, X, y, cv=CV)
    matriz_confusion = confusion_matrix(y, predicciones)
    #reporte = classification_report(y, predicciones)
    #precision, recall, f1_score, _ = reporte.split('\n')[-2].split()[1:]#
    
    return ((matriz_confusion))

In [35]:
modelos_all= []
class_names = []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_cross_val_predict(exp))   
    class_names = exp['Key'].unique()

conf_mat_exp1 = pd.DataFrame(modelos_all[0], index=class_names, columns=class_names)
conf_mat_exp2 = pd.DataFrame(modelos_all[1], index=class_names, columns=class_names)
conf_mat_exp3 = pd.DataFrame(modelos_all[2], index=class_names, columns=class_names)
conf_mat_exp4 = pd.DataFrame(modelos_all[3], index=class_names, columns=class_names)


In [36]:
display (conf_mat_exp1)
display (conf_mat_exp2)
display (conf_mat_exp3)
display (conf_mat_exp4)

,4.0,0.0,1.0,3.0,2.0
4.0,30,2,3,2,2
0.0,0,22,0,0,0
1.0,0,0,23,0,0
3.0,0,0,0,22,0
2.0,1,0,0,0,23


,4.0,0.0,1.0,3.0,2.0
4.0,26,2,3,2,6
0.0,2,25,0,0,0
1.0,2,0,25,0,0
3.0,1,0,0,28,0
2.0,2,0,0,0,27


,4.0,0.0,1.0,3.0,2.0
4.0,35,9,4,3,3
0.0,4,41,0,0,0
1.0,1,0,30,0,0
3.0,3,0,0,28,0
2.0,4,1,0,0,29


,4.0,0.0,1.0,3.0,2.0
4.0,31,9,5,2,10
0.0,5,42,0,0,0
1.0,6,0,26,0,0
3.0,8,0,0,23,0
2.0,5,0,0,0,36


# Regresion Linear   N/A

In [37]:

def evaluar_linear_regresion(dataframe):
    
    y = dataframe['Key'].values
    X = dataframe.drop(columns=['Key']).values
        
    model_LINEAR.fit(X, y)

    evaluacion = model_LINEAR.score(X, y)
    
    return ((evaluacion))


In [38]:
modelos_all= []

for exp in experimentos_nor:
    
    modelos_all.append(evaluar_linear_regresion(exp))   

datos_linear_regresion = pd.DataFrame(modelos_all, columns=['Linear_regresion'])
#creo los valores nuevo para los indices de las filas 
nuevos_indices = {0: 'Exp_1', 1: 'Exp_2', 2: 'Exp_3', 3: 'Exp_4'}
# combio los valores anteriores en el dataframene con rename
datos_linear_regresion = datos_linear_regresion.rename(index=nuevos_indices)

NameError: name 'model_LINEAR' is not defined

In [39]:
display(datos_linear_regresion)

NameError: name 'datos_linear_regresion' is not defined

In [ ]:


# probar regresion logistica

